In [1]:
import json
import pandas as pd
import os
import shutil

In [2]:
DATA_DIR = "../../data"
DATA_FILE = os.path.join(DATA_DIR, "data.json")
DUMP_FILE = os.path.join(DATA_DIR, "dump.pkl")

In [3]:
store_columns = (
    "id",  # 음식점 고유번호
    "store_name",  # 음식점 이름
    "branch",  # 음식점 지점 여부
    "area",  # 음식점 위치
    "tel",  # 음식점 번호
    "address",  # 음식점 주소
    "latitude",  # 음식점 위도
    "longitude",  # 음식점 경도
    "category",  # 음식점 카테고리
)

review_columns = (
    "id",  # 리뷰 고유번호
    "store",  # 음식점 고유번호
    "user",  # 유저 고유번호
    "score",  # 평점
    "content",  # 리뷰 내용
    "reg_time",  # 리뷰 등록 시간
)

user_columns = (
    "id", # 유저 고유번호
    "gender", # 유저 성별
    "birth_year", # 유저 나이
)

menu_columns = (
    "id", # 메뉴 고유번호
    "store",  # 음식점 고유번호
    "menu_name", # 메뉴 이름
    "price" # 메뉴 가격
)

def import_data(data_path=DATA_FILE):
    """
    Req. 1-1-1 음식점 데이터 파일을 읽어서 Pandas DataFrame 형태로 저장합니다
    """

    try:
        with open(data_path, encoding="utf-8") as f:
            data = json.loads(f.read())
    except FileNotFoundError as e:
        print(f"`{data_path}` 가 존재하지 않습니다.")
        exit(1)

    stores = []  # 음식점 테이블
    reviews = []  # 리뷰 테이블
    users = {} # 유저 테이블
    menus = []
    idx = 0
    r_idx = 0 # reviews의 id를 사용하면 중복되는 것이 있기 때문에
    for d in data:

        categories = [c["category"] for c in d["category_list"]]
        stores.append(
            [
                d["id"],
                d["name"],
                d["branch"],
                d["area"],
                d["tel"],
                d["address"],
                d["latitude"],
                d["longitude"],
                "|".join(categories),
            ]
        )
        
        for review in d["review_list"]:
            r = review["review_info"]
            u = review["writer_info"]
            r_idx +=1
            reviews.append(
                [r_idx, d["id"], u["id"], r["score"], r["content"], r["reg_time"]]
            )
            try:
                users[u["id"]]
            except KeyError:
                user_colums = (
                    "id", # 유저 고유번호
                    "gender", # 유저 성별
                    "birth_year", # 유저 나이
                )
                users.update({u["id"]: [u["id"], u["gender"], u["born_year"]]})

        for menu in d["menu_list"]:
            idx += 1
            menus.append(
                [
                    idx,
                    d["id"],
                    menu["menu"],
                    menu["price"]
                ]
            )

    store_frame = pd.DataFrame(data=stores, columns=store_columns)
    review_frame = pd.DataFrame(data=reviews, columns=review_columns)
    user_frame = pd.DataFrame(data=list(users.values()), columns=user_columns)
    menu_frame = pd.DataFrame(data=menus, columns=menu_columns)
    # 하단 값 계산 후 반영하는 것들
    menu_frame.loc[menu_frame['price'].isna(),'price']=0
    
    return {"stores": store_frame, "reviews": review_frame, "users" : user_frame, "menus" : menu_frame} 

In [4]:
def dump_dataframes(dataframes):
    pd.to_pickle(dataframes, DUMP_FILE)

In [5]:
def load_dataframes():
    return pd.read_pickle(DUMP_FILE)

### def main
##### (jupyter 상에서 사용하기위해 def형태 제거)

##### data.json에서  데이터 파싱

In [6]:
print("[*] Parsing data...")
data = import_data()
print("[+] Done")

[*] Parsing data...
[+] Done


#### 생성된 데이터프레임 파일을 pkl 로 저장

In [7]:
print("[*] Dumping data...")
dump_dataframes(data)
print("[+] Done\n")

[*] Dumping data...
[+] Done



#### pkl에서 데이터 불러오기

In [8]:
data = load_dataframes()
#출력 형식 위한 정의
term_w = shutil.get_terminal_size()[0] - 1
separater = "-" * term_w

#### 각 데이터프레임의 헤드(5개) 출력

In [9]:
print("[음식점]")
print(f"{separater}\n")
print(data["stores"].head())
print(f"\n{separater}\n\n")

[음식점]
-------------------------------------------------------------------------------------------------------

   id     store_name branch  area            tel                address  \
0   1           Agal   None    홍대  010-6689-5886   서울특별시 마포구 동교동 170-13   
1   2         Assisy   None    광주   062-367-0700    광주광역시 서구 농성동 631-33   
2   3  Battered Sole   None   이태원    02-749-6867   서울특별시 용산구 이태원동 118-9   
3   4      Chakyoung   None  달맞이길   051-756-5566  부산광역시 해운대구 중2동 1509-5   
4   5       Delabobo   None   발산역   02-2667-9854      서울특별시 강서구 등촌동 689   

    latitude   longitude   category  
0  37.556862  126.926666   아구찜|포장마차  
1  35.150746  126.890062         카페  
2  37.535032  126.991664    피쉬앤칩스|펍  
3  35.158587  129.175004  레스토랑|카프레제  
4  37.559904  126.840512  디저트카페|디저트  

-------------------------------------------------------------------------------------------------------




In [10]:
print("[리뷰]")
print(f"{separater}\n")
print(data["reviews"].head())
print(f"\n{separater}\n\n")

[리뷰]
-------------------------------------------------------------------------------------------------------

   id  store    user  score  \
0   1     15   68632      5   
1   2     18  389728      5   
2   3     19   68716      4   
3   4     37  774353      2   
4   5     38  115682      3   

                                             content             reg_time  
0         전포 윗길에 새로 생긴! 호주에서 온 쉐프가 직접 요리하는 호주식 레스토랑!  1970-01-01 00:00:00  
1                    샌드위치 내용물도 알차게 들어있고 맛있었어요 가성비 추천  1970-01-01 00:00:00  
2  홈플러스 1층 매장 푸드코트 내 있는 매장인데 계란찜정식은 처음보고 시켜봣는데 사진...  1970-01-01 00:00:00  
3  전 여기 5년전에 가봤었는데 그때 기억은 별로였어요\n단체손님이 많았고, 차려지는건...  1970-01-01 00:00:00  
4  친구들끼리 술 간단하게마시러 감. 스끼다시 괜찮지만 회 양이 조금 부족한 느낌. 맛...  2019-03-15 22:16:47  

-------------------------------------------------------------------------------------------------------




In [11]:
print("[유저]")
print(f"{separater}\n")
print(data["users"].head())
print(f"\n{separater}\n\n")

[유저]
-------------------------------------------------------------------------------------------------------

       id gender birth_year
0   68632      남       1990
1  389728      여       1993
2   68716      여       1984
3  774353      여       1972
4  115682      남       1991

-------------------------------------------------------------------------------------------------------




In [12]:
print("[메뉴]")
print(f"{separater}\n")
print(data["menus"].head())
print(f"\n{separater}\n\n")

[메뉴]
-------------------------------------------------------------------------------------------------------

   id  store             menu_name    price
0   1      3  메인 (피쉬앤칩스, 오리 가슴살 등)  14000.0
1   2      3          스타터 보드(파테 등)  18000.0
2   3      3        디저트 (초콜렛 퐁당 등)   8500.0
3   4      3               생과일 칵테일   9000.0
4   5      3                   생맥주   7000.0

-------------------------------------------------------------------------------------------------------




***
### 구조 파악을 위한 출력
#### 기존 데이터를 django model로 변환하는 과정에 생기는 오류(Value error)등을 방지하기 위해 확인

In [13]:
 stores = data['stores']  # //stores는 문제없이 동작하므로 패스~~
menus = data['menus']
reviews = data['reviews']
users = data['users']

In [ ]:
menus.dtypes

In [15]:
#menu의 price중에 NaN값이 1개가 발견되었고, 0으로 넣어줌
menus.loc[menus['price'].isna(),'price']=0

In [ ]:
reviews.dtypes

In [17]:
reviews['id'].value_counts()

2047     1
82621    1
6806     1
4759     1
27288    1
        ..
21856    1
23905    1
17762    1
19811    1
2049     1
Name: id, Length: 91398, dtype: int64

In [20]:
users.dtypes

id             int64
gender        object
birth_year    object
dtype: object

In [21]:
users[users['birth_year'].isna()]

,id,gender,birth_year


In [22]:
users['id'].value_counts()

245752    1
256517    1
864883    1
75903     1
318968    1
         ..
480258    1
565280    1
910630    1
133586    1
294912    1
Name: id, Length: 18992, dtype: int64